In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [2]:
url = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo-zona-norte/'
c = requests.get(url).content

In [3]:
soup = BeautifulSoup(c)

## Criando uma lista com os preços dos alugueis

In [4]:
lista_precos_tags = soup.find_all('span', {'class':'price-tag-fraction'})

- For loop para obter a lista de preços

In [5]:
lista_precos = []
for tag in lista_precos_tags:
    lista_precos.append(tag.text)
lista_precos[0:5]

['950', '600', '1.299', '1.000', '590']

 - List comprehension

In [6]:
# for loop utilizando list compreehension
lista_precos  = [tag.text for tag in lista_precos_tags]
lista_precos[0:5]

['950', '600', '1.299', '1.000', '590']

## Criando uma lista com a área e quantidade de quartos

In [7]:
lista_area_quartos_tags = soup.find_all('ul', {'class':'ui-search-card-attributes'})

In [8]:
lista_area_quartos = [info.text for info in lista_area_quartos_tags ]
lista_area_quartos[0:5]

['42 m² construídos2 quartos',
 '70 m² construídos1 quarto',
 '80 m² construídos2 quartos',
 '50 m² construídos1 quarto',
 '30 m² construídos1 quarto']

## Criando uma lista com os endereços

In [9]:
lista_enderecos_tags = soup.find_all('span' , {'class':'ui-search-item__group__element'})

In [10]:
lista_enderecos = [end.text for end in lista_enderecos_tags]
lista_enderecos = lista_enderecos[1::2]
lista_enderecos[:5]

['Parque Peruche, São Paulo Zona Norte, São Paulo',
 'Nobrega De Souza Coutinho 05, Jardim Peri, São Paulo Zona Norte, São Paulo',
 'Rua José Eduardo Raduan 5, Parque Edu Chaves, São Paulo Zona Norte, São Paulo',
 'Rua Campo Belo Do Sul 113, Parque Vitória, São Paulo Zona Norte, São Paulo',
 'Rua Rodolfo Bardela 335, Brasilândia, São Paulo Zona Norte, São Paulo']

## Criando  função que retorna um DataFrame com as informaçoes obtidas anteriormente

In [11]:
zonas = ['norte', 'sul', 'leste', 'oeste']
URL='https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', 
           '_Desde_241', '_Desde_289']

In [12]:
def scrap(url):
    c = requests.get(url).content
    soup = BeautifulSoup(c)
    lista_precos_tags = soup.find_all('span', {'class':'price-tag-fraction'})
    lista_precos  = [tag.text for tag in lista_precos_tags]
    lista_area_quartos_tags = soup.find_all('ul', {'class':'ui-search-card-attributes'})
    lista_area_quartos = [info.text for info in lista_area_quartos_tags ]
    lista_enderecos_tags = soup.find_all('span' , {'class':'ui-search-item__group__element'})
    lista_enderecos = [end.text for end in lista_enderecos_tags]
    lista_enderecos = lista_enderecos[1::2]
    df=convert_df(zona, lista_area_quartos, lista_enderecos, lista_precos)
    return df


def convert_df(zona, lista_area_quartos, lista_enderecos, lista_precos):
    zonas_lista = [zona]*len(lista_precos)
    data = {'zona': zonas_lista,
            'area_quartos': lista_area_quartos,
            'enderecos_lista':lista_enderecos,
            'aluguel': lista_precos}
    df = pd.DataFrame(data)
    return df

In [15]:
lista_dfs = []
for zona in zonas:
    for pag in paginas:
        url='https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-{}/{}'.format(zona,pag)
        print(url)
        df=scrap(url)
        lista_dfs.append(df)
        sleep(1)

https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_145
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_193
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_241
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_289
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_193
ht

In [18]:
df = pd.concat(lista_dfs)

In [19]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel
0,norte,42 m² construídos2 quartos,"Parque Peruche, São Paulo Zona Norte, São Paulo",950
1,norte,75 m² construídos2 quartos,"Avenida Nova Cantareira, Santana, São Paulo Zo...",1.272
2,norte,58 m² construídos2 quartos,"Avenida Deputado Emílio Carlos, Limão, São Pau...",1.100
3,norte,68 m² construídos2 quartos,"Rua General José De Almeida Botelho, Parque Ma...",981
4,norte,80 m² construídos3 quartos,"Rua Alcides Godói, Mandaqui, São Paulo Zona No...",1.768


In [22]:
df.to_csv('dados_aluguels_imoveis_ml_sao_paulo.csv', index =False , encoding = 'latin-1')